# Generazione Grafici

## Media differenza (x Challenge)

In [6]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Percorsi di base
uniqueness_base_path = './results/uniqueness/real/'
instability_base_path = './results/instability/real/'
graphs_path = './results/graph/xChallenge'

# Creare la cartella dei grafici se non esiste
os.makedirs(graphs_path, exist_ok=True)

# Ottenere tutte le cartelle disponibili in uniqueness
uniqueness_dirs = [d for d in os.listdir(uniqueness_base_path) if os.path.isdir(os.path.join(uniqueness_base_path, d))]

# Lista per memorizzare le differenze globali
differences_list = []

for uniqueness_dir in uniqueness_dirs:
    uniqueness_path = os.path.join(uniqueness_base_path, uniqueness_dir)
    instability_path = os.path.join(instability_base_path, uniqueness_dir)  # Assumiamo stesso nome
    
    if not os.path.exists(instability_path):
        print(f"Cartella corrispondente non trovata per {uniqueness_dir}, saltata.")
        continue
    
    # Estrarre il nome della configurazione
    config_name = uniqueness_dir.split('_')[0] if '_' in uniqueness_dir else uniqueness_dir
    
    # Trova i file CSV nella cartella uniqueness
    uniqueness_files = [f for f in os.listdir(uniqueness_path) if f.endswith('.csv')]
    instability_files = [f for f in os.listdir(instability_path) if f.endswith('.csv')]
    
    if not instability_files:
        print(f"Nessun file trovato nella cartella instability per {uniqueness_dir}, saltata.")
        continue
    
    # Carica e aggrega tutti i file di instability
    instability_dfs = [pd.read_csv(os.path.join(instability_path, f)) for f in instability_files]
    instability_combined_df = pd.concat(instability_dfs)
    instability_avg = instability_combined_df.groupby('n-th challenge')['dist'].mean()
    
    for uq_file in uniqueness_files:
        uniqueness_df = pd.read_csv(os.path.join(uniqueness_path, uq_file))
        
        # Aggrega i dati calcolando la media delle distanze per ciascun 'n-th challenge'
        uniqueness_avg = uniqueness_df.groupby('n-th challenge')['dist'].mean()
    
        
        # Calcolare la differenza assoluta tra uniqueness e instability
        diff = (uniqueness_avg - instability_avg)
        total_diff = diff.mean()
        differences_list.append((config_name, uq_file, total_diff))
        
        print(f"Differenza totale per {config_name} - {uq_file}: {total_diff}")
        
        # Creare il grafico
        plt.figure(figsize=(10, 6))
        plt.plot(uniqueness_avg.index, uniqueness_avg.values, label='Uniqueness', color='blue', marker='o')
        plt.plot(instability_avg.index, instability_avg.values, label='Instability (media)', color='red', marker='x')
        
        # Aggiungere titolo e etichette
        uq_file = uq_file.removeprefix("unique_table_")
        plt.title(f'Confronto tra Uniqueness e Instability\nCircuito: {config_name} - Configurazione: {uq_file}')
        plt.xlabel('n-th Challenge')
        plt.ylabel('Media della Distanza')
        plt.legend()
        plt.grid(True)
        
        graph_filename = os.path.join(graphs_path, f"{config_name}_{uq_file.replace('.csv', '.png')}")
        
        if not os.path.exists(graph_filename):
            plt.savefig(graph_filename)
            print(f"Grafico salvato: {graph_filename}")
        else:
            print(f"Il file {graph_filename} esiste già. Nessuna sovrascrittura effettuata.")
        
        plt.close()

# Stampare le configurazioni con maggiore differenza totale
print("\nTop 100 configurazioni con maggiore differenza totale:")
differences_list.sort(key=lambda x: x[2], reverse=True)
for i, (config, file, diff) in enumerate(differences_list[:100]):
    print(f"{i+1}. {config} - {file}: {diff}")


Differenza totale per saturn - unique_table_nq10_nc10_s10000_nr5.csv: 0.24316033333333203
Grafico salvato: ./results/graph/xChallenge/saturn_nq10_nc10_s10000_nr5.png
Differenza totale per uranus - unique_table_nq10_nc10_s5000_nr5.csv: 0.25913933333333194
Grafico salvato: ./results/graph/xChallenge/uranus_nq10_nc10_s5000_nr5.png
Differenza totale per mars - unique_table_nq10_nc10_s10000_nr5.csv: 0.12321366666666608
Grafico salvato: ./results/graph/xChallenge/mars_nq10_nc10_s10000_nr5.png
Differenza totale per jupiter - unique_table_nq12_nc10_s5000_nr5.csv: 0.028501999999999368
Grafico salvato: ./results/graph/xChallenge/jupiter_nq12_nc10_s5000_nr5.png
Differenza totale per saturn - unique_table_nq8_nc10_s20000_nr5.csv: 0.20121399999999978
Grafico salvato: ./results/graph/xChallenge/saturn_nq8_nc10_s20000_nr5.png
Differenza totale per saturn - unique_table_nq12_nc20_s20000_nr5.csv: 0.24180049999999476
Grafico salvato: ./results/graph/xChallenge/saturn_nq12_nc20_s20000_nr5.png
Differenza 

## Differenza tra min(unicità) e max(Instability)

In [7]:
import os
import pandas as pd
import matplotlib.pyplot as plt

# Percorsi di base
uniqueness_base_path = './results/uniqueness/real/'
instability_base_path = './results/instability/real/'
graphs_path = './results/graph/globalDiff'

# Ottenere tutte le cartelle disponibili in uniqueness
uniqueness_dirs = [d for d in os.listdir(uniqueness_base_path) if os.path.isdir(os.path.join(uniqueness_base_path, d))]

# Lista per memorizzare le differenze globali
differences_list = []

for uniqueness_dir in uniqueness_dirs:
    uniqueness_path = os.path.join(uniqueness_base_path, uniqueness_dir)
    instability_path = os.path.join(instability_base_path, uniqueness_dir)
    
    if not os.path.exists(instability_path):
        print(f"Cartella corrispondente non trovata per {uniqueness_dir}, saltata.")
        continue
    
    config_name = uniqueness_dir.split('_')[0] if '_' in uniqueness_dir else uniqueness_dir
    
    uniqueness_files = [f for f in os.listdir(uniqueness_path) if f.endswith('.csv')]
    instability_files = [f for f in os.listdir(instability_path) if f.endswith('.csv')]
    
    if not instability_files:
        print(f"Nessun file trovato nella cartella instability per {uniqueness_dir}, saltata.")
        continue
    
    # Carica e unisci tutti i dati di instability
    instability_dfs = [pd.read_csv(os.path.join(instability_path, f)) for f in instability_files]
    instability_combined_df = pd.concat(instability_dfs)
    max_instability = instability_combined_df['dist'].max()
    

    for uq_file in uniqueness_files:
        uniqueness_df = pd.read_csv(os.path.join(uniqueness_path, uq_file))
        min_uniqueness = uniqueness_df['dist'].min()

        uq_file = uq_file.removeprefix("unique_table_")

        total_diff = min_uniqueness - max_instability
        differences_list.append((config_name, uq_file, total_diff))
        
        #print(f"---------- {config_name}_{uq_file} ----------")
        #print(f"MIN uniqueness({min_uniqueness}) - MAX instability({max_instability}) = {total_diff}")
        #print("-----Fine----")
        
# Stampare le 10 configurazioni con maggiore differenza globale
print("\nTop 10 configurazioni con maggiore differenza globale (min uniqueness - max instability):")
differences_list.sort(key=lambda x: x[2], reverse=True)
for i, (config, file, diff) in enumerate(differences_list[:110]):
    print(f"{i+1}. {config}_{file}: {diff}")


Top 10 configurazioni con maggiore differenza globale (min uniqueness - max instability):
1. uranus_nq8_nc20_s20000_nr5.csv: 0.22724999999999998
2. uranusTitania_nq10_nc10_s20000_nr5.csv: 0.21849999999999867
3. uranusTitania_nq8_nc10_s20000_nr5.csv: 0.2169999999999997
4. uranusTitania_nq10_nc20_s20000_nr5.csv: 0.2038999999999997
5. uranusTitania_nq8_nc20_s20000_nr5.csv: 0.19765
6. uranusTitania_nq8_nc20_s20000_nr5.csv: 0.19765
7. uranus_nq12_nc20_s20000_nr5.csv: 0.19749999999999338
8. uranusTitania_nq8_nc10_s10000_nr5.csv: 0.19339999999999957
9. uranusTitania_nq10_nc10_s10000_nr5.csv: 0.19319999999999898
10. uranusTitania_nq8_nc10_s5000_nr5.csv: 0.17519999999999988
11. saturn_nq8_nc20_s20000_nr5.csv: 0.1629499999999998
12. saturn_nq10_nc20_s20000_nr5.csv: 0.1577500000000006
13. uranusTitania_nq12_nc10_s20000_nr5.csv: 0.1575999999999901
14. uranus_nq10_nc20_s10000_nr5.csv: 0.1529999999999989
15. uranus_nq10_nc20_s5000_nr5.csv: 0.1465999999999997
16. saturn_nq12_nc20_s20000_nr5.csv: 0.1

## Stampo i grafici con le 3 curve insieme (instability, randomness e uniqueness)

In [9]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Percorsi di base
uniqueness_base_path = './results/uniqueness/real/'
instability_base_path = './results/instability/real/'
randomness_base_path = './results/randomness/real/'
graphs_path = './results/graph/Combined'

# Creare la cartella dei grafici se non esiste
os.makedirs(graphs_path, exist_ok=True)

# Ottenere tutte le cartelle disponibili in uniqueness
uniqueness_dirs = [d for d in os.listdir(uniqueness_base_path) if os.path.isdir(os.path.join(uniqueness_base_path, d))]

for uniqueness_dir in uniqueness_dirs:
    uniqueness_path = os.path.join(uniqueness_base_path, uniqueness_dir)
    instability_path = os.path.join(instability_base_path, uniqueness_dir)  # Assumiamo stesso nome
    randomness_path = os.path.join(randomness_base_path, uniqueness_dir)  
    
    if not os.path.exists(instability_path):
        print(f"Cartella corrispondente non trovata per {uniqueness_dir}, saltata.")
        continue

    if not os.path.exists(randomness_path):
        print(f"Cartella corrispondente non trovata per {uniqueness_dir}, saltata.")
        continue
    
    # Estrarre il nome della configurazione
    config_name = uniqueness_dir.split('_')[0] if '_' in uniqueness_dir else uniqueness_dir
    
    # Trova i file CSV nella cartella uniqueness
    uniqueness_files = [f for f in os.listdir(uniqueness_path) if f.endswith('.csv')]
    randomness_files = [f for f in os.listdir(randomness_path) if f.endswith('.csv')]
    instability_files = [f for f in os.listdir(instability_path) if f.endswith('.csv')]
    
    if not instability_files:
        print(f"Nessun file trovato nella cartella instability per {uniqueness_dir}, saltata.")
        continue

    if not randomness_files:
        print(f"Nessun file trovato nella cartella randomness per {uniqueness_dir}, saltata.")
        continue
    
    # Carica e aggrega tutti i file di instability
    instability_dfs = [pd.read_csv(os.path.join(instability_path, f)) for f in instability_files]
    instability_combined_df = pd.concat(instability_dfs)
    instability_avg = instability_combined_df.groupby(['n-th challenge', 'i-th execution', 'j-th execution'])['dist'].mean().reset_index()

     # Carica e aggrega tutti i file di randomness
    randomness_dfs = [pd.read_csv(os.path.join(randomness_path, f)) for f in randomness_files]
    randomness_combined_df = pd.concat(randomness_dfs)
    randomness_avg = randomness_combined_df.groupby(['n-th challenge', 'm-th challenge'])['dist'].mean().reset_index()

    for uq_file in uniqueness_files:
        uniqueness_df = pd.read_csv(os.path.join(uniqueness_path, uq_file))
        
        # Creare il grafico
        plt.figure(figsize=(10, 6))
        sns.kdeplot(data=instability_avg, x='dist', label='Instability')
        sns.kdeplot(data=uniqueness_df, x='dist', label='Uniqueness')
        sns.kdeplot(data=randomness_avg, x='dist', label='Randomness')

        # Aggiungere titolo e etichette
        uq_file = uq_file.removeprefix("unique_table_")
        config_name = config_name.capitalize()
        plt.title(f'Rappresentazione metriche \n Circuito: {config_name} - Configurazione: {uq_file}')
        plt.legend()
        plt.grid(True)
        
        # Definire il nome del file di output
        graph_filename = os.path.join(graphs_path, f"{config_name}_{uq_file.replace('.csv', '.png')}")
        
        if not os.path.exists(graph_filename):
            plt.savefig(graph_filename)
            print(f"Grafico salvato: {graph_filename}")
        else:
            print(f"Il file {graph_filename} esiste già. Nessuna sovrascrittura effettuata.")
        
        plt.close()

Il file ./results/graph/Combined/Saturn_nq10_nc10_s10000_nr5.png esiste già. Nessuna sovrascrittura effettuata.
Il file ./results/graph/Combined/Uranus_nq10_nc10_s5000_nr5.png esiste già. Nessuna sovrascrittura effettuata.
Il file ./results/graph/Combined/Mars_nq10_nc10_s10000_nr5.png esiste già. Nessuna sovrascrittura effettuata.
Il file ./results/graph/Combined/Jupiter_nq12_nc10_s5000_nr5.png esiste già. Nessuna sovrascrittura effettuata.
Il file ./results/graph/Combined/Saturn_nq8_nc10_s20000_nr5.png esiste già. Nessuna sovrascrittura effettuata.
Il file ./results/graph/Combined/Saturn_nq12_nc20_s20000_nr5.png esiste già. Nessuna sovrascrittura effettuata.
Il file ./results/graph/Combined/Uranustitania_nq8_nc20_s20000_nr5.png esiste già. Nessuna sovrascrittura effettuata.
Il file ./results/graph/Combined/Uranus_nq12_nc10_s10000_nr5.png esiste già. Nessuna sovrascrittura effettuata.
Il file ./results/graph/Combined/Earth_nq8_nc20_s20000_nr5.png esiste già. Nessuna sovrascrittura effe

### Stampo le metriche per le 3 configurazioni

In [8]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Percorsi di base
uniqueness_base_path = './results/uniqueness/real/'
randomness_base_path = './results/randomness/real/'
instability_base_path = './results/instability/real/'

# Ottenere tutte le cartelle disponibili in uniqueness
uniqueness_dirs = [d for d in os.listdir(uniqueness_base_path) if os.path.isdir(os.path.join(uniqueness_base_path, d))]

for uniqueness_dir in uniqueness_dirs:
    uniqueness_path = os.path.join(uniqueness_base_path, uniqueness_dir)
    instability_path = os.path.join(instability_base_path, uniqueness_dir)  # Assumiamo stesso nome
    randomness_path = os.path.join(randomness_base_path, uniqueness_dir)  # Assumiamo stesso nome
    
    if not os.path.exists(instability_path):
        print(f"Cartella corrispondente non trovata per {uniqueness_dir}, saltata.")
        continue

    if not os.path.exists(randomness_path):
        print(f"Cartella corrispondente non trovata per {uniqueness_dir}, saltata.")
        continue
    
    
    # Estrarre il nome della configurazione
    config_name = uniqueness_dir.split('_')[0] if '_' in uniqueness_dir else uniqueness_dir
    
    # Trova i file CSV nella cartella uniqueness
    uniqueness_files = [f for f in os.listdir(uniqueness_path) if f.endswith('.csv')]
    instability_files = [f for f in os.listdir(instability_path) if f.endswith('.csv')]
    randomness_files = [f for f in os.listdir(randomness_path) if f.endswith('.csv')]
    
    if not instability_files:
        print(f"Nessun file trovato nella cartella instability per {uniqueness_dir}, saltata.")
        continue

    if not randomness_files:
        print(f"Nessun file trovato nella cartella randomness per {uniqueness_dir}, saltata.")
        continue
    
    # Carica e aggrega tutti i file di instability
    instability_dfs = [pd.read_csv(os.path.join(instability_path, f)) for f in instability_files]
    instability_combined_df = pd.concat(instability_dfs)
    # Calcola la media della colonna 'dist'
    mean_inta = instability_combined_df['dist'].mean()

    randomness_dfs = [pd.read_csv(os.path.join(randomness_path, f)) for f in randomness_files]
    randomness_combined_df = pd.concat(randomness_dfs)
    # Calcola la media della colonna 'dist'
    mean_rand = randomness_combined_df['dist'].mean()
    
    for uq_file in uniqueness_files:
        uniqueness_df = pd.read_csv(os.path.join(uniqueness_path, uq_file))
        mean_unique = uniqueness_df['dist'].mean()
    uq_file = uq_file.removeprefix("unique_table_")
    print(f"{config_name}_{uq_file} | Intability : {mean_inta}")
    print(f"{config_name}_{uq_file} | Randomness : {mean_rand}")
    print(f"{config_name}_{uq_file} | Uniqueness : {mean_unique}")

saturn_nq10_nc10_s10000_nr5.csv | Intability : 0.1541263333333329
saturn_nq10_nc10_s10000_nr5.csv | Randomness : 0.6345762962962935
saturn_nq10_nc10_s10000_nr5.csv | Uniqueness : 0.39728666666666496
uranus_nq10_nc10_s5000_nr5.csv | Intability : 0.21140066666666613
uranus_nq10_nc10_s5000_nr5.csv | Randomness : 0.6401599999999967
uranus_nq10_nc10_s5000_nr5.csv | Uniqueness : 0.4705399999999981
mars_nq10_nc10_s10000_nr5.csv | Intability : 0.1204529999999996
mars_nq10_nc10_s10000_nr5.csv | Randomness : 0.7974822222222174
mars_nq10_nc10_s10000_nr5.csv | Uniqueness : 0.24366666666666567
jupiter_nq12_nc10_s5000_nr5.csv | Intability : 0.30107799999999457
jupiter_nq12_nc10_s5000_nr5.csv | Randomness : 0.8499185185184983
jupiter_nq12_nc10_s5000_nr5.csv | Uniqueness : 0.329579999999994
saturn_nq8_nc10_s20000_nr5.csv | Intability : 0.07169433333333329
saturn_nq8_nc10_s20000_nr5.csv | Randomness : 0.5723348148148145
saturn_nq8_nc10_s20000_nr5.csv | Uniqueness : 0.27290833333333303
saturn_nq12_nc20_